## 1. Импорт  библиотек

In [1]:
import random
import math
import copy

import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

import os
import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

import torch
from kan import KAN

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
#%load_ext autotime

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_27680\581288377.py:26: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
from tools import KAN_es
from algos import create_XY_data, alg_keras_mlp, vector_pred_NN, alg_KAN_model, vector_pred_KAN, multi_exp

## 3. Обучение нейронных сетей

### 3.0. Загрузка наборов данных

In [3]:
'''
initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_vld.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_vld.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_vld.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
'''
# !Some troubles with *.csv naming!

initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_pro.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_pro.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_pro.csv")

In [4]:
data_col_names = initial_trn_data_udus.columns

### 3.1. Шкалирование данных

In [5]:
mm_scaler = MinMaxScaler()
mm_scaler.fit(pd.concat([initial_trn_data_fdfs,
                         initial_trn_data_udfs, 
                         initial_trn_data_udus],
                        axis=0, sort=False, ignore_index=True))

MinMaxScaler()

In [6]:
'''
scaled_trn_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names) 
scaled_vld_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names)
scaled_tst_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names)

scaled_trn_data_udfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_udfs),columns=data_col_names)
scaled_vld_data_udfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_udfs),columns=data_col_names)
scaled_tst_data_udfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_udfs),columns=data_col_names)

scaled_trn_data_udus = pd.DataFrame(mm_scaler.transform(initial_trn_data_udus),columns=data_col_names)
scaled_vld_data_udus = pd.DataFrame(mm_scaler.transform(initial_vld_data_udus),columns=data_col_names)
scaled_tst_data_udus = pd.DataFrame(mm_scaler.transform(initial_tst_data_udus),columns=data_col_names)
'''
trn = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names) 
vld = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names)
tst = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names)


### Мульти-эксперимент

In [7]:
l_algos_names=['GMT_KAN_hn_1', 'GMT_KAN_hn_2', 'GMT_KAN_hn_3', 'GMT_KAN_hn_4', 'GMT_KAN_hn_5', 
               'GMT_KAN_hn_6', 'GMT_KAN_hn_7', 'GMT_KAN_hn_8', 'GMT_KAN_hn_9', 'GMT_KAN_hn_10']

l_algos=[vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN,
         vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN]

Data = (trn, vld, tst)

mult_data = [Data for i in range(10)]

l_geophysical_method = ['GMT' for i in range(10)]

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(10)]

l_kwargs=[{'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':2, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':3, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':4, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':5, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':6, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':7, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':8, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':9, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':10, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          ]

l_metrics_names=['mae', 'rmse', 'r2', 'mape']

num_iter=5

In [8]:
full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

--- Processing GMT_KAN_hn_1
iter: 1


trn_ls: 6.10e-02 | vl_ls: 6.27e-02 | e_stop: 25/25 | tst_ls: 6.09e-02 | reg: 3.24e+01 :   4%|   | 38/1000 [01:21<34:16,  2.14s/it]


Early stopping criteria raised


trn_ls: 1.63e-01 | vl_ls: 1.68e-01 | e_stop: 25/25 | tst_ls: 1.60e-01 | reg: 5.11e+01 :  12%|▏ | 117/1000 [04:19<32:35,  2.21s/it]


Early stopping criteria raised


...

trn_ls: 1.63e-01 | vl_ls: 1.70e-01 | e_stop: 25/25 | tst_ls: 1.59e-01 | reg: 3.11e+01 :   8%|▏  | 83/1000 [03:16<36:12,  2.37s/it]


Early stopping criteria raised


trn_ls: 3.08e-01 | vl_ls: 3.11e-01 | e_stop: 25/25 | tst_ls: 3.05e-01 | reg: 2.54e+01 :   3%|   | 26/1000 [01:04<40:02,  2.47s/it]

Early stopping criteria raised
-------



c:\Users\Gavriil\VS_projects\KAN_geo\.venv\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [9]:
full_df

,alg_name,iter,mae,rmse,r2
0,GMT_KAN_hn_1,1,0.14007,0.17543,0.64241
1,GMT_KAN_hn_1,2,0.13545,0.17022,0.65990
2,GMT_KAN_hn_1,3,0.13684,0.17176,0.65548
3,GMT_KAN_hn_1,4,0.13375,0.16864,0.66194
4,GMT_KAN_hn_1,5,0.13062,0.16459,0.67304
5,GMT_KAN_hn_2,1,0.13964,0.17535,0.64265
6,GMT_KAN_hn_2,2,0.14016,0.17544,0.64409
7,GMT_KAN_hn_2,3,0.13972,0.17585,0.64022
8,GMT_KAN_hn_2,4,0.13650,0.17147,0.65573
9,GMT_KAN_hn_2,5,0.13235,0.16684,0.66684


In [10]:
full_df.to_excel('full_metrics_3.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [11]:
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel('aggr_metrics_3.xlsx')
aggr_df

mae                rmse                  r2          
                   mean       std      mean       std      mean       std
alg_name                                                                 
GMT_KAN_hn_1   0.135346  0.003516  0.170128  0.003990  0.658554  0.011104
GMT_KAN_hn_10  0.136922  0.004401  0.172018  0.005120  0.652232  0.016499
GMT_KAN_hn_2   0.137674  0.003315  0.172990  0.003869  0.649906  0.011191
GMT_KAN_hn_5   0.137676  0.004428  0.172974  0.005050  0.649946  0.014508